In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Preprocessing

### Qx Table

In [ ]:
df_Qx = pd.read_excel('Template.xlsx', sheet_name="위험률", header = 3).fillna(0)
print(df_Qx.shape)
df_Qx.head()

In [ ]:
# 구분자
cols = ['상해급수', '운전자급수']

# 위험률키
def getRiskKey(riskCode, injure, driver):
    return f"{riskCode}_{injure}_{driver}" 

In [ ]:
# 위험률키
riskKeys = []
for row in df_Qx[['위험률코드'] + cols + ['x', '남자', '여자']].values:
    rCode, injure, driver, x, male, female = row
    rKey = getRiskKey(rCode, int(injure), int(driver))
    riskKeys.append(rKey)

df_Qx['위험률키'] = riskKeys
df_Qx.head()

Qx Dictionary

key : 성별코드_위험률코드_상해급수_운전자급수 

value : ${q_0}, {q_1}, ... {q_{119}}$

In [ ]:
riskKeys = set(riskKeys)
print(f"위험률키 개수 : {len(riskKeys)}")

Qx_dict = {}

for rKey in tqdm(riskKeys):
    qx_male, qx_female = [0.]*120, [0.]*120
    df_q = df_Qx.loc[df_Qx['위험률키'] == rKey][['x', '남자', '여자']]
    for x, male, female in df_q.values:
        if x == "ZZ":   # 단일률
            qx_male = [male for _ in range(120)]
            qx_female = [male for _ in range(120)]
            break
        else:
            qx_male[int(x)] = male
            qx_female[int(x)] = female
    Qx_dict[f"{1}_{rKey}"] = qx_male
    Qx_dict[f"{2}_{rKey}"] = qx_female

### Code Table

In [ ]:
df_Code = pd.read_excel('Template.xlsx', sheet_name="코드", header = 3).fillna("")
print(df_Code.shape)
df_Code.head()

In [ ]:
df_Code = df_Code[['담보키', '급부번호',\
       '탈퇴율코드', '탈퇴율Type', '부담보기간',\
              '급부율코드', '급부율Type', '지급률', '감액률', '감액기간', \
                     '납면율코드', '납면율Type', '면책기간']]
print(df_Code.shape)
df_Code.head()

In [ ]:
covKeys = np.unique(df_Code['담보키'].values)
print(f"담보키 개수 : {len(covKeys)}개")

Coverage Dictionary

- key : 담보키

- value : 담보정보

    - 탈퇴

    - 급부

    - 납면

    - 급부개수

    - 결합위험률 적용여부

    - 단일률 적용여부

In [ ]:
Coverage_dict = {}

for covKey in tqdm(covKeys):

    coverage_info = {}
    df_c = df_Code[df_Code['담보키'] == covKey]
    numBenefit = 0      # 급부개수
    useCombQx = False   # 결합위험률 적용 여부
    useSingleQx = False # 단일률 적용 여부

    for row in df_c.values:
        _, bNum, eCode, eType, nCov, bCode, bType, pRate, rRate, rPeriod, gCode, gType, iPeriod = row

        if (eType == "C") | (bType == "C") | (gType == "C"):
            useCombQx = True
        if (eType == "S") | (bType == "S") | (gType == "S"):
            useSingleQx = True
        
        if bNum == 99:  # 납입면제 (grant)
            coverage_info[99] = {
                "GxCode" : gCode,   # 납면율 code
                "GxType" : gType,   # 납면율 type
                "InvalidPeriod" : 0 if iPeriod == "" else int(iPeriod)   # 무효해지기간
            }

        else:
            coverage_info[bNum] = {}

            coverage_info[bNum]['Exit'] = {
                "ExCode" : eCode,   # 탈퇴율 code
                "ExType" : eType,   # 탈퇴율 type
                "NonCov" : 0 if nCov == "" else int(nCov)     # 부담보
            }

            if bNum != 0:
                coverage_info[bNum]['Benefit'] = {
                    "BxCode" : bCode,   # 급부율 code
                    "BxType" : bType,   # 급부율 type
                    "PayRate" : 1. if pRate == "" else float(pRate),          # 지급률
                    "ReducePeriod" : 0 if rPeriod == "" else int(rPeriod),   # 감액기간
                    "ReduceRate" : 0. if rRate == "" else float(rRate)      # 감액률
                }
                numBenefit += 1
    
    coverage_info["NumBenefit"] = numBenefit    # 급부개수
    coverage_info["UseSingleQx"]= useSingleQx   # 단일률적용여부
    coverage_info["UseCombQx"] = useCombQx      # 결합위험률적용여부


    Coverage_dict[covKey] = coverage_info

### Comb Table

In [ ]:
df_Comb = pd.read_excel("Template.xlsx", sheet_name="결합위험률", header=3).fillna("")
print(df_Comb.shape)
df_Comb.head()

In [ ]:
df_Comb = df_Comb[['담보키', '결합위험률코드', 'Operation', '위험률개수'] \
    + [f"위험률코드({i})" for i in range(1, 8+1)] \
        + [f"제외기간({i})" for i in range(1, 8+1)]]
df_Comb.head()

In [ ]:
covKeys_comb = np.unique(df_Comb['담보키'].values)
print(f"결합위험률 적용 담보키 개수 : {len(covKeys_comb)}개")

In [ ]:
for covKey in covKeys_comb:
    comb_info = {}
    df_c = df_Comb.loc[df_Comb['담보키'] == covKey]
    for row in df_c.values:
        _, combRiskKey, oper, nRiskKey = row[:4]
        rCodes = row[4:4+8][:nRiskKey]
        periods = row[12:12+8][:nRiskKey]
        periods = [0 if p=="" else int(p) for p in periods]
        comb_info[combRiskKey] = {
            "Operation" : int(oper),
            "RiskCodes" : rCodes,
            "Periods" : periods
        }
    Coverage_dict[covKey]["CombInfo"] = comb_info

In [ ]:
Coverage_dict["LC0738"]

# Qx mapping

In [ ]:
def getQx(sex : int, riskCode : str, injure : int, driver : int):
    riskKey = f"{sex}_{riskCode}_{injure}_{driver}"
    return Qx_dict[riskKey]

In [ ]:
test = getQx(sex=1, riskCode="HKFM221", injure = 0, driver = 0)
print(test[:5])

In [ ]:
def getCombQxDict(comb_info : dict, sex : int, x : int):

    result = {}    
    for (combRiskKey, info) in comb_info.items():
        oper = info["Operation"]
        rCodes = info["RiskCodes"]
        periods = info["Periods"]

        numRiskCode = len(rCodes)
        qx = [[0.]*120]*numRiskCode

        for i in range(numRiskCode):

            if rCodes[i] in result.keys():
                qx[i] = result[rCodes[i]]
            else:
                qx[i] = getQx(sex = sex, riskCode = rCodes[i], injure = 0, driver = 0)    

        if oper == 1:

            combQx = [0.]*120
            for i in range(numRiskCode):
                for t in range(120):
                    if t == x:
                        combQx[t] += qx[i][t] * (1-periods[i]/12)
                    else:
                        combQx[t] += qx[i][t] 


        elif oper == 2:

            combQx = [1.]*120
            for i in range(numRiskCode):
                for t in range(120):
                    if t == x:
                        combQx[t] *= 1-qx[i][t]* (1-periods[i]/12)
                    else:
                        combQx[t] *= 1-qx[i][t] 
            for t in range(120):
                combQx[t] = 1-combQx[t]

        else:
            raise Exception("결합위험률 계산방법이 이상허게 들어옴")
        
        result[combRiskKey] = combQx

    return result

In [ ]:
test = getCombQxDict(comb_info = Coverage_dict["LC0738"]['CombInfo'], sex = 1, x= 40, n = 5)
test["C1"][40:45]

In [ ]:
def QxMapping(covKey : str, sex:int, x:int, n:int, injure : int, driver : int):
    coverage_info = Coverage_dict[covKey]

    numBenefit = coverage_info['NumBenefit']
    Ex = [[0.]*(n+1)]*(numBenefit+1)
    Bx = [[0.]*(n+1)]*(numBenefit+1)

    if coverage_info['UseCombQx']:
        combQxDict = getCombQxDict(coverage_info['CombInfo'], sex, x)
        
    for i in range(numBenefit+1):
        eCode = coverage_info[i]['Exit']['ExCode']
        eType = coverage_info[i]['Exit']['ExType']
        if eType == "C":
            Ex[i] = combQxDict[eCode][x:x+n+1]
        else:
            Ex[i] = getQx(sex = sex, riskCode=eCode, injure = injure, driver = driver)[x:x+n+1]

        if i>0:
            bCode = coverage_info[i]['Benefit']['BxCode']
            bType = coverage_info[i]['Benefit']['BxType']

            if bType == "C":
                Bx[i] = combQxDict[bCode][x:x+n+1]
            else:
                Bx[i] = getQx(sex = sex, riskCode=bCode, injure = injure, driver = driver)[x:x+n+1]

    
    gCode = coverage_info[99]['GxCode']
    gType = coverage_info[99]['GxType']
    if gType == "C":
        Gx = combQxDict[gCode][x:x+n+1]
    else:
        Gx = getQx(sex = sex, riskCode=gCode, injure=injure,driver=driver)[x:x+n+1]

    return Ex, Bx, Gx

In [ ]:
Ex, Bx, Gx = QxMapping(covKey="LA1568", sex = 1, x = 40, n = 5, injure=0, driver= 0)
print(f"Ex(0) : {Ex[0]}")
print(f"Ex(1) : {Ex[1]}")
# print(f"Bx(0) : {Bx[0]}")
print(f"Bx(1) : {Bx[1]}")
print(f"Gx : {Gx}")

# Calculation

In [ ]:
Coverage_dict["LA3348_1"]

In [ ]:
def Calulation(Ex : list, Bx : list, Gx : list, numBenefit : int, \
    nonCov : list, payRate : list, reduceRate : list, reducePeriod : list, invalidPeriod : list, \
        sex:int, x:int, n:int, injure:int, driver:int, v:int):
    

    # init
    lx = [[0.]*(n+1)]*(numBenefit+1)
    lxPrime = [0.]*(n+1)
    Nx = [0.]*(n+1)
    NxPrime = [0.]*(n+1)
    Cx = [[0.]*(n+1)]*(numBenefit+1)
    Mx = [[0.]*(n+1)]*(numBenefit+1)
    SUMx = [0.]*(n+1)

    # lx
    for i in range(numBenefit+1):
        lx[i][0] = 100000
        for t in range(n):
            if t == 0:
                lx[i][t+1] = lx[i][t]*(1-Ex[i][t] * (1-nonCov[i]/12))   # 부담보
            else:
                lx[i][t+1] = lx[i][t]*(1-Ex[i][t])
    # Dx
    Dx = [lx[0][t]*v**t for t in range(n+1)]
    # Nx
    for t in range(n)[::-1]:
        Nx[t] = Nx[t+1] + Dx[t]
        
    
    # l'x
    lxPrime[0] = 100000
    for t in range(n):
        if t==0:
            lxPrime[t+1] = lxPrime[t]*(1-Gx[t] * (1-invalidPeriod/12))
        else:
            lxPrime[t+1] = lxPrime[t]*(1-Gx[t])
    # D'x
    DxPrime = [lxPrime[t]*v**t for t in range(n+1)]
    # N'x
    for t in range(n)[::-1]:
        NxPrime[t] = NxPrime[t+1] + DxPrime[t]
        

    # Cx
    for i in range(1, numBenefit+1):
        Cx[i] = [lx[i][t]*Bx[i][t]*v**(t+0.5) for t in range(n+1)]

    # Mx
    for i in range(1, numBenefit+1):
        for t in range(n)[::-1]:
            Mx[i][t] = Cx[i][t] + Mx[i][t+1]

    # SUMx
    for i in range(1, numBenefit+1):
        pRate = payRate[i]
        rPeriod = reducePeriod[i]
        rRate = reduceRate[i]
        for t in range(n+1):
            if t<reducePeriod[i]:
                SUMx[t] += pRate * ((1-rRate) * (Mx[t]-Mx[x+rPeriod]) + \
                    (Mx[x+rPeriod] - Mx[n]))
            else:
                SUMx[t] += pRate * (Mx[t] - Mx[n])

    return {
        'lx' : lx,
        "l'x" : lxPrime,
        "Dx" : Dx,
        "D'x" : DxPrime,
        "Nx" : Dx,
        "N'x" : DxPrime,
        "Cx" : Cx,
        "Mx" : Mx,
        "SUMx" : SUMx
    }   